## Crawl Articles

In [9]:
import csv
import os

def get_csv_reader(filename, delimiter):
    reader = []
    if not os.path.isfile(filename):
        csvfile = open(filename,"w")
    else:
        csvfile = open(filename, "rt")
        reader = csv.DictReader(csvfile, delimiter=delimiter)
        
    return list(reader)

In [50]:
#CSV파일을 dict의 객체형태로 읽어온다.(증권리포트 데이터 입력시사용)

with open('report_test.csv','r') as f:
    reader = csv.DictReader(f)
   # for row in reader:
    #    print(row)
        
article = {}
article['title'] = row['title']
article['text'] = row['text']
article['url'] = row['url']

print(article)
articles = []
for x in article:
    articles.append(article)

{'title': '유통/화장품 2Q19 Earnings Preview - 과도한 변동성에도 양호한 펀더멘탈은 존재', 'text': '? Top-picks\n\n- 유통: SK네트웍스, GS리테일, 현대홈쇼핑, 호텔신라\n- 화장품: 클리오, 코스맥스, LG생활건강\n\n▶ 백화점, 가전전문점\n\n- 4월 백화점 기존점 감소에도 2분기 전체적으로 수입 명품 카테고리의 두 자리 수 고신장세 지속. 리빙, 식품, 아동스포츠, 롱패딩 등 여성복의 개선을 대신. 백화점 3사 기존점은 3~5% 예상.\n- 가전판매(가전전문점) 2016년 이후 2018년 상반기까지 신규 가전 카테고리의 보급률 확대가 정점을 찍음. 특히 가전 카테고리는 2017년 하반기 수요 급증을 나타내며 2018년 3분기와 4분기 기저 부담 작용\n? 면세점:\n2019년 전자상거래법의 점진적 반영에도 상위 3사의 경쟁 우위도 독보적. 반면에 관세청의 국산 면세품 표시제 의무화 제시 등에 따른 불확실성 부담\n? 할인점:\n2분기 기존점 성장률 -5%~-2% 예상. 과도한 새벽 배송 경쟁에 따른 이마트의 2분기 영업적자 가능성 제시. 여기에 재산세 반영에 따른 고정비 부담\n? 렌탈:\n웅진코웨이의 재매각에 따른 단기 불확실성 부각에도 오히려 웅진그룹 리스크에 대한 확실한 제거 기대. 한편 SK네트웍스는 2분기 기존 사업에서의 실적 개선과 웅진코웨이 인수 관심으로 긍정적 센티먼트 확산\n? 홈쇼핑:\n소형가전, 렌탈 판매 호조와 패션 부문의 회복으로 취급고 증가 전망. IPTV송출수수료 인상은 여전히 20%~30% 수준이 불가피하지만 위성 및 케이블에서 하향 노력\n? 편의점:\n2019년 점포 순증 BGF리테일 700점(17년 1,600점대), GS리테일 700점 가까이로 마감. 공정위가 제시한 담배권 거리제한 영향으로 2019년 상위 편의점의 점포 순증도 500점 이하로 예상되는 반면 기존점의 개선도 가능할 전망.\n▶ 화장품\n\n\n? 클리오, LG생활건강:\n\nLG생활건강은 보수적인 가

In [10]:
#증권리포트 분석을 위해 csv파일 읽어올때 사용하는 부분
articles =[]
articles = get_csv_reader("report_test.csv",",")

print(articles[0])

OrderedDict([('title', '유통/화장품 - 루이비통그룹(크리스찬디올) 19년 상반기 실적 점검(김영민)'), ('text', '? 19.1H LVMH[MC FP] 연결(yoy) 매출액 15.3%, 순이익 9.2%\n\n? 화장품 부문 매출액 +9% yoy, 계속사업이익 +6% yoy (Organic Growth 기준)\n\n? 아시아, 미국, 유럽 지역 호조\n(크리스챤 디올 꾸뛰르 실적 개선, LVMH: 향수, 화장품 플래그쉽브랜드 급속 성장, 세포라: 매장, 온라인 부문 매출 성장 강세, DFS(면세사업부문): 국제 관광객 증가에 기인)'), ('url', 'https://www.ibks.com/index.do')])


In [24]:
import re
from newspaper import Article, ArticleException

#기사 링크를 입력으로 받아 제목, 내용 그리고 링크를 파싱하여 dictionary 형태로 반환하는 함수
#input - 기사 url
#output - 기사의 dictionary 형태
def crawl_newspaper(url):
    
    try:
        article = Article(url,
                          language='ko',
                          memorize_articles=False,
                          fetch_images=False,
                          skip_bad_cleaner=True)
        article.download()
        article.parse()

        article_data = {}
        article_data['title'] = article.title.strip()  # 기사 제목
        article_data['text'] = article.text.replace("\n", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
        article_data['url'] = url  # 기사 링크

        return article_data
    except ArticleException as AE:
        raise AE
    

In [25]:
import requests
from bs4 import BeautifulSoup

#네이버에서 검색어를 이용해 해당 기간 내의 뉴스 리스트를 반환하는 함수
#input - 검색어, 기사일자, 뉴스리스트 시작페이지
#output - 뉴스 리스트
def get_newspaper_list(query,  # 검색어
                       date=None,  # 기사 일자
                       start_page=1  # 뉴스 리스트 페이지
                      ):
    
    url = 'https://search.naver.com/search.naver?&where=news&query={query}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={start_date}&de={end_date}&start={start_page}'  # query와 기간을 조건으로 한 검색 url 
    if not date:
        date = datetime.now()  # 오늘 날짜 instance 생성
        date = date.strftime("%Y.%m.%d")  # 날자 instance를 문자열 형태로 변환 ex) 2019.06.30
    
    res = requests.get(url.format(query=query, 
                                  start_date=date, 
                                  end_date=date, 
                                  start_page=(start_page - 1) * 10 + 1))
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.text, 'lxml')
    
    # 최대 기사 페이지 수를 확인
    pages = [tag.get_text() for tag in soup.select(".paging > a, .paging > strong")] # 이전페이지,1,2,3페이지
    max_page = 0
    for page in pages:
        try:
            page = int(page)
            max_page = max(page, max_page)
        except:
            continue
            
    if start_page > max_page:  # 더이상 페이지가 없을 경우 빈 리스트를 반환
        return []
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출
    link_list = [a['href'] for a in soup.select(".news > ul.type01 > li > dl > dt > a")]  
    return link_list

In [26]:
from datetime import datetime, timedelta
from time import time

#get_newspaper_list 함수를 이용하여 시작일자 ~ 끝일자까지의 뉴스리스트를 반환
#input - 검색어 키워드, 검색 시기, 검색 종기, 검색 최대 페이지 수
#output - 검색 시기~종기 기간의 검색된 뉴스리스트 반환

def crawl_newspaper_by_query(query,  # 검색어 키워드
                             start_date=None,
                             end_date=None,
                             max_page=10): #원래10 임시로 1
    
    
    article_list = []  # 기사 리스트
    crawled_url = set()  # 검색한 기사 중복 검색을 위한 변수
        
    if not start_date:
        start_date = datetime.now()  # 오늘 날짜 instance 생성
    else:
        start_date = datetime.strptime(start_date, "%Y.%m.%d")  # 2019.06.30과 같은 문자열을 python instance로 변경
        
    if not end_date:
        end_date = datetime.now()
    else:
        end_date = datetime.strptime(end_date, "%Y.%m.%d")
    
    while start_date < end_date:
        print("Date: ", end_date)
        for cur_page in range(1, max_page):
            print("Page: {}".format(cur_page))
            
            # end_date 날짜의 기사 링크 수집
            link_list = get_newspaper_list(query, end_date.strftime("%Y.%m.%d"), start_page=cur_page)

            if not link_list:  # 더이상 수집할 뉴스가 없을 경우
                break
            
            for link in link_list:
                # 중복 수집 방지
                if link in crawled_url:
                    continue
                else:
                    crawled_url.update([link])
                
                try:
                    article = crawl_newspaper(link)
                    article['date'] = start_date
                    article_list.append(article)
                except ArticleException as AE:
                    continue
        
        end_date = end_date - timedelta(days=1)  # 수집할 날짜를 하루 전으로 변경
    return article_list

In [27]:
#뉴스기사 크롤링 실행
#input - 검색어, 검색시기, 검색종기, 검색할 최대페이지수
#output - 검색된 뉴스리스트 articles에 저장
#예시data articles[0] 출력
start = time()
articles = crawl_newspaper_by_query("유통", "2019.07.16", "2019.07.26", max_page=10)
print(time() - start)
print("Total articles: {:,}".format(len(articles)))
print("="*20, "Article sample", "="*20)
print(articles[0])

Date:  2019-07-26 00:00:00
Page: 1
5.47629976272583
Total articles: 10
==================== Article sample ====================
{'title': '국내 유통된 중국산 조개젓서 A형간염 유전자 검출', 'text': '[아시아경제 오주연 기자] 중국산 조개젓에서 A형간염 바이러스 유전자가 검출됐다.  26일 질병관리본부에 따르면 인천시 보건환경연구원을 통해 중국에서 제조하고 국내에서 추가 가공한 조개젓에서 A형간염 바이러스 유전자가 검출됐다.  질병관리본부는 충남 소재 병원 종사자 6명에서 A형간염이 생긴 것을 인지하고 충청남도와 관할 보건소와 함께 현장 역학조사를 하는 과정에서 해당 조개젓이 병원 직원식당에 제공된 것을 확인하고 개봉하지 않은 조개젓 식품을 수거해 검사했다.  식품의약품안전처는 A형 간염바이러스 유전자가 검출된 해당 제품을 회수, 폐기하고 질병관리본부 등과 함께 환자와 식품과의 인과관계 등을 조사할 예정이다.  한편 조개젓에서 A형간염 바이러스 유전자가 검출된 것은 이번이 여섯 번째다.  A형 간염에 걸리면 발열, 오한, 오심, 구토, 황달 등의 증상이 있을 수 있다. 의심 증상이 나타나면 즉시 의사에게 진료를 받아야 한다.  오주연 기자 moon170@asiae.co.kr', 'url': 'https://view.asiae.co.kr/article/2019072619220973837', 'date': datetime.datetime(2019, 7, 25, 0, 0)}


In [54]:
#import pandas as pd    #pandas를 불러온다.


#크롤링 결과 results를 pandas의 DataFrame 형식으로 읽어온다.
#data = pd.DataFrame(articles)
#data.head()
#data.to_csv('crawl_articles.csv')

## Text preprocessing

In [30]:
#형태소 분류작업 (정규화, 어근화작업도 수행)
#input - 형태소 분류할 문장
#output - 분류된 형태소
#예시data 출력
#from konlpy.tag import Okt

# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
#tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)

#sentence = "SK텔레콤-LG전자, 5G로 클라우드와 로봇 결합했다… 로봇산업 혁신 기대"
#words = tagger.pos(sentence, 
#                   norm=True,  # 정규화 - 입니다 ㄹㄹ -> 입니다로 모아주는
#                   stem=True)  # 어근화 - 원형으로 바꾸는 작업
#for w, t in words:
#    print(w, t)

    
#"""
#정규화 (normalization): 입니닼ㅋㅋ => 입니다 ㅋㅋ
#어근화 (stemming): 한국어를 처리하는 예시입니다 => 한국어, 를, 처리, 하다, 예시, 이다
#"""

'\n정규화 (normalization): 입니닼ㅋㅋ => 입니다 ㅋㅋ\n어근화 (stemming): 한국어를 처리하는 예시입니다 => 한국어, 를, 처리, 하다, 예시, 이다\n'

In [28]:
#형태소 분류작업 (정규화, 어근화작업도 수행)
#input - 형태소 분류할 문장
#output - 분류된 형태소
#예시data 출력
from konlpy.tag import Okt
from collections import Counter
# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)

sentence = "SK텔레콤-LG전자, 5G로 클라우드와 로봇 결합했다… 로봇산업 혁신 기대"
words = tagger.pos(sentence, 
                   norm=True,  # 정규화 - 입니다 ㄹㄹ -> 입니다로 모아주는
                   stem=True)  # 어근화 - 원형으로 바꾸는 작업
#for w, t in words:
#    print(w, t)
    
"""
정규화 (normalization): 입니닼ㅋㅋ => 입니다 ㅋㅋ
어근화 (stemming): 한국어를 처리하는 예시입니다 => 한국어, 를, 처리, 하다, 예시, 이다
"""

#키워드 추출 조건 설정
#input - 키워드로 포함하고자하는 품사 입력, 키워드로 포함하지 않을 금지단어 입력
#output - 기사갯수 100단위로 출력, 시data 첫번째 기사항목 출력
words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
stopwords = set(["글자"])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..


for i, article in enumerate(articles):
    if i % 100 == 0:
        print(i)
        
    words = tagger.pos(article['text'], norm=True, stem=True)
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    vocab.update(tmp_words)
    words_list.append((tmp_words, article))
    


vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    
    # 기사 내 서로 다른 단어 개수가 10개 이상인 기사만 사용    
    if len(words) > 10:
        new_words_list.append((words, article))

0


## Build term-document matrix

In [29]:
#input - words_list, vocab
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

print(len(words_list))
print(len(vocab))
term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix)

10
1032
Term document matrix shape: (10, 1032)
  (0, 1022)	0.0756664335641109
  (0, 1019)	0.0756664335641109
  (0, 1015)	0.06432338116935418
  (0, 1012)	0.05003282556053471
  (0, 1010)	0.06432338116935418
  (0, 988)	0.05627535822106181
  (0, 976)	0.10006565112106942
  (0, 971)	0.05003282556053471
  (0, 942)	0.0756664335641109
  (0, 914)	0.040619876393048086
  (0, 878)	0.0756664335641109
  (0, 877)	0.0756664335641109
  (0, 866)	0.0756664335641109
  (0, 832)	0.2269993006923327
  (0, 826)	0.0756664335641109
  (0, 824)	0.05003282556053471
  (0, 806)	0.1513328671282218
  (0, 805)	0.0756664335641109
  (0, 797)	0.0756664335641109
  (0, 796)	0.0756664335641109
  (0, 780)	0.0756664335641109
  (0, 776)	0.11255071644212362
  (0, 774)	0.3783321678205545
  (0, 773)	0.05627535822106181
  (0, 772)	0.0756664335641109
  :	:
  (9, 91)	0.11233668090522352
  (9, 88)	0.05136084690412453
  (9, 87)	0.025680423452062263
  (9, 85)	0.030209016063967187
  (9, 73)	0.030209016063967187
  (9, 72)	0.0302090160639671

## Topic modeling

In [30]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition
from sklearn.decomposition import NMF
 
K = 5  # topic 개수지정
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (10, 5)
H shape: (5, 1032)


In [31]:
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')
        #vocabWrite[index] = vocab[index] + ' '
        
    print("\n", "="*20)

0th topic
리츠 점포 홈플러스 롯데 사업 매장 스페셜 온라인 부동산 상품 기존 사원 판촉 배송 물류 고객 통해 사장 이마트 마트 
1th topic
화환 유통 통관 화훼 홍보 센터 aT 린지 위촉 올해 캠퍼스 뉴데일리 소비 식품 대사 월드 개발 교수 수출 한국 
2th topic
간염 조개 유전자 바이러스 질병 본부 관리 식품 오주연 병원 증상 조사 해당 기자 추가 가공 인천 연구원 제조 역학 
3th topic
퓨리 당질 전자랜드 일자리 딤채 스킨 마스크 LED 위니 푸르밀 본점 제품 환경 판매 IH 압력밥솥 대한민국 최초 조성 출시 
4th topic
양파 마늘 이지은 장아찌 행사 직접 미래세 대인 농업인 농협 농협유통 엄용수 어린이 경남 수산 하나로마트 양재 이웃 로서 경상남도 


In [11]:
new_topic_words = []

for index in H[0].argsort()[::-1][:]:
    print(vocab[index], end=' ')

NameError: name 'H' is not defined

In [41]:
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    print("{}th topic".format(k))
    for index in W[:, k].argsort()[::-1][:10]:
        print(words_list[index][1]['title'])
    print("\n", "="*20)

0th topic
유통/화장품 - 루이비통그룹(크리스찬디올) 19년 상반기 실적 점검






1th topic





유통/화장품 - 루이비통그룹(크리스찬디올) 19년 상반기 실적 점검

2th topic





유통/화장품 - 루이비통그룹(크리스찬디올) 19년 상반기 실적 점검

3th topic





유통/화장품 - 루이비통그룹(크리스찬디올) 19년 상반기 실적 점검

4th topic





유통/화장품 - 루이비통그룹(크리스찬디올) 19년 상반기 실적 점검



## Visualization

In [42]:
from sklearn.manifold import TSNE

print("W shape:", W.shape)
tsne = TSNE(n_components=2, init='pca', verbose=1) 
W2d = tsne.fit_transform(W)
topic_index = [v.argmax() for v in W]  # 각각의 문서가 어느 topic에 속해 있는지 (hard clustering)


W shape: (6, 5)
[t-SNE] Computing 5 nearest neighbors...
[t-SNE] Indexed 6 samples in 0.000s...
[t-SNE] Computed neighbors for 6 samples in 0.001s...
[t-SNE] Computed conditional probabilities for sample 6 / 6
[t-SNE] Mean sigma: 1125899906842624.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: -112.601036
[t-SNE] KL divergence after 1000 iterations: -13.810756


In [43]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [44]:
tools_to_show = "hover,box_zoom,pan,save,reset,wheel_zoom"
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article['title'] for words, article in new_words_list],
    'topic': [str(i) for i in topic_index],
    'color': [Category20[K][i] for i in topic_index]
})



p.circle('x', 'y', source=source, legend='topic', color='color')

p.legend.location = 'top_left'
hover = p.select({'type': HoverTool})
hover.tooltips = [('Topic', '@topic'), ('id', '@id'), ('Article', '@document')]
hover.mode = 'mouse'

show(p)

In [49]:
#topic과 Keyword 분포, 빈도를 확인할 수 있는 visualization
#jupiter note에서 사용할때 그래프 확인후 interrupt the kernel로 종료 후 다음진행 필요
from sklearn.preprocessing import normalize

def y_to_doc_topic(y):
    n_topics = y.shape[1]
    base = 1 / n_topics
    doc_topic_prob = normalize(y, norm='l1')
    rowsum = doc_topic_prob.sum(axis=1)
    doc_topic_prob[np.where(rowsum == 0)[0]] = base
    return doc_topic_prob


In [50]:
def components_to_topic_term(components):
    n_terms = components.shape[1]
    base = 1 / n_terms
    topic_term_prob = normalize(components, norm='l1')
    rowsum = topic_term_prob.sum(axis=1)
    topic_term_prob[np.where(rowsum == 0)[0]] = base
    return topic_term_prob

In [53]:
doc_topic_prob = y_to_doc_topic(W)
topic_term_prob = components_to_topic_term(H)
doc_lengths = np.asarray(term_doc_matrix.sum(axis=1)).reshape(-1)
term_frequency = np.asarray(term_doc_matrix.sum(axis=0)).reshape(-1)

In [54]:
from pyLDAvis import prepare, show
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn


print(topic_term_prob.shape)
print(len(vocab))
prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    vocab,
    term_frequency,
    R = 10, #num of displayed terms,
    sort_topics = False
)

#display(prepared_data)
show(prepared_data)

(5, 40)
40


C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


TypeError: Object of type complex is not JSON serializable

## Siple search engine

In [57]:
query = '삼성'
query_index = word2id[query]  # SKT 단어의 index

query_vec = term_doc_matrix[:, query_index]  # 각 문서에서 SKT 단어에 해당하는 값을 추출
nonzero_doc = query_vec.nonzero()[0]  # 0이 아닌 문서들만 사용
print('논제로_독', nonzero_doc);
ranks = query_vec.toarray().flatten().argsort()[::-1]
print('랭크', ranks);

for r in ranks[:10]:
    if r in nonzero_doc:
        print(new_words_list[r][1]['title'], new_words_list[r][1]['url'])


쿼리_벡   (0, 0)	0.015525654762480539
  (3, 0)	0.03038726997479271
  (4, 0)	0.047614238050700815
  (5, 0)	0.037838814854266684
  (6, 0)	0.07245710725661934
  (8, 0)	0.025748012811725146
  (14, 0)	0.021595244832185844
  (15, 0)	0.09776399273998984
  (16, 0)	0.02429739287382223
  (18, 0)	0.017462641751393698
  (19, 0)	0.01473374374899782
  (20, 0)	0.06411222382086657
  (24, 0)	0.21356202330529828
  (30, 0)	0.09302662522908386
  (36, 0)	0.05507677467104858
  (37, 0)	0.030252790532069262
  (38, 0)	0.04922516548982161
  (40, 0)	0.03068573869154204
  (41, 0)	0.07648189033898505
  (43, 0)	0.02224612903100561
  (48, 0)	0.030037437748755526
  (49, 0)	0.16509858674155767
  (51, 0)	0.013680973709323037
  (52, 0)	0.0198230950745455
  (55, 0)	0.03610805928367062
  :	:
  (2378, 0)	0.2680643708044099
  (2381, 0)	0.06229570752933079
  (2383, 0)	0.03808180434867973
  (2384, 0)	0.2443522077720839
  (2386, 0)	0.28721141887442675
  (2387, 0)	0.10444358604809643
  (2390, 0)	0.024845760774294696
  (2391, 0)	0.